## Batch features

Exports (RPM limit = False / True):
```
    - SHAFT, A, fault
    - SHAFT, B, fault
    - BEARINGS, A, fault
    - BEARINGS, B, fault

    - SHAFT, A, anomaly, 0.6
    - SHAFT, B, anomaly, 0.6
    - BEARINGS, A, anomaly, 0.6
    - BEARINGS, B, anomaly, 0.6

    - SHAFT, A, anomaly, 0.9
    - SHAFT, B, anomaly, 0.9
    - BEARINGS, A, anomaly, 0.9
    - BEARINGS, B, anomaly, 0.9
```

In [ ]:
PLACEMENTS = [['ax', 'ay', 'az'], ['bx', 'by', 'bz']]
TARGETS = ['fault', 'anomaly']
ANOMALY_SEVERITIES = [0.6, 0.9]
SHAFT_FAULTS = {'normal': 'N', 'imbalance': 'I', 'horizontal-misalignment': 'HM', 'vertical-misalignment': 'VM'}
BEARING_FAULTS = {'overhang-cage_fault': 'O-Cage', 'underhang-cage_fault': 'U-Cage',
                  'underhang-ball_fault': 'U-Ball', 'overhang-ball_fault': 'O-Ball',
                  'underhang-outer_race': 'U-Race', 'overhang-ball_fault': 'O-Race'}
FAULT_TYPES = [SHAFT_FAULTS, BEARING_FAULTS]


CORR_SIGNIFICANT = 0.95
TRAIN_SIZE = 0.8


# Variables
PLACE = PLACEMENTS[1]
VAR_TARGET = TARGETS[1]
ANOMALY_SEVERITY = ANOMALY_SEVERITIES[1]       # If it is anomaly

FAULT_CLASSES = FAULT_TYPES[1]
RPM_LIMIT = False                         # False, True
BALANCE = True                           # False, True

In [ ]:
import os
import itertools
from collections import Counter
import sys
sys.path.append('../')
from vibrodiagnostics import selection, discovery, models

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gmean, spearmanr, kendalltau

from typing import Callable, Dict, Tuple, List
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import silhouette_score

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

from river import feature_selection, stream, preprocessing


PATH_PREFIX = '../../datasets/'
FEATURES_PATH =  os.path.join(PATH_PREFIX, 'features_data')

TD_FD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_AND_FREQ_FEATURES_PATH)
TD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_FEATURES_PATH)
FD_FEATURES = os.path.join(FEATURES_PATH, selection.FREQ_FEATURES_PATH)

METRICS_TITLES = ('Correlation', 'F statistic', 'Mutual information')
METRICS_ONLINE = (selection.Correlation, selection.FisherScore, selection.MutualInformation)
METRICS_OFFLINE = (selection.corr_classif, f_classif, mutual_info_classif)

Offline feature selection

In [ ]:
def calculate_scores(x: pd.DataFrame, y: pd.DataFrame, metric: Callable) -> pd.DataFrame:
    scores = metric(x, y)
    if isinstance(scores, tuple):
        scores = scores[0]
    leaderboard = (
        pd.DataFrame(zip(x.columns, scores), columns=['feature', 'score'])
        .set_index('feature')
        .sort_values(by='score', ascending=False)
    )
    return leaderboard


def features_subset_offline(
        filename: str, 
        classes: Dict[str, str],
        axis: Tuple[str], 
        label: str,
        train_size: float, 
        anomaly_severity: float,
        balance: bool,
        rpm_limit: bool
    ):
    features = pd.read_csv(filename).fillna(0)
    features = features[features['fault'].isin(classes)]
    if rpm_limit:
        RPM = 2900
        RPM_RANGE = 500
        features = features[features['rpm'].between(RPM - RPM_RANGE, RPM + RPM_RANGE, inclusive='both')]
    
    features = models.fault_labeling(features, classes, anomaly_severity)

    columns = features.columns.str.startswith(tuple(axis))
    X = features[features.columns[columns]].fillna(0)
    y = features[label].astype('category')

    # Balance dataset
    if balance:
        oversample = RandomOverSampler(sampling_strategy='not majority', random_state=10)
        X, y = oversample.fit_resample(X, y.to_numpy())
        X.reset_index(drop=True, inplace=True)
        y = pd.Series(y)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_size, stratify=y, random_state=10
    )
    return X_train, y_train, X_test, y_test


def axis_to_magnitude(x: pd.DataFrame, axis: list, window_size: int):
    if isinstance(window_size, int):
        x = x.loc[:,x.columns.str.endswith(f'_{window_size}')]
        x.columns = x.columns.str.extract(r'([\w\_]+)_(\w+)$')[0]

    feature_names = x.columns.str.extract(r'([a-z]{2})_([\w\_\-]+)')[1].unique()

    result = pd.DataFrame()
    for name in feature_names:
        vector_dims = [f'{dim}_{name}' for dim in axis]
        result[name] = x[vector_dims].apply(np.linalg.norm, axis=1)

    return result


def load_feature_set(feature_set: str, axis: list, window_size: int, online=False) -> tuple:
    if online:
        func = features_subset_online
    else:
        func = features_subset_offline

    x_train, y_train, x_test, y_test = func(
        feature_set,
        FAULT_CLASSES,
        axis,
        VAR_TARGET,
        TRAIN_SIZE,
        ANOMALY_SEVERITY,
        BALANCE,
        RPM_LIMIT
    )
    x_train_mag = axis_to_magnitude(x_train, axis, window_size)
    x_test_mag = axis_to_magnitude(x_test, axis, window_size)

    return x_train_mag, y_train, x_test_mag, y_test


def feature_ranking(feature_set: str, window_size=None):
    axis = PLACE
    metric_ranks = pd.DataFrame()

    for metric_name, metric in zip(('corr', 'f_stat', 'mi'), METRICS_OFFLINE):
        x, y, _, _ = load_feature_set(feature_set, axis, window_size)
        scores = calculate_scores(x, y, metric)
        metric_ranks[metric_name] = scores

    return metric_ranks


def corr_among_features(feature_set, axis, window_size=None):
    x, y, _, _ = load_feature_set(feature_set, axis, window_size)

    correlations = [
        {'feature_1': k[0], 'feature_2': k[1], 'corr': v}
        for k, v in x.corr().abs().stack().to_dict().items()
        if k[0] != k[1]
    ]
    correlations = pd.DataFrame.from_records(correlations).sort_values(by='corr', ascending=False)
    correlations[correlations['corr'] > 0.7]
    return correlations


def extract_best_features(x_train, y_train, best_features, corr, n=3) -> List[str]:
    # Remove correlated features independent of tuple order
    correlations = corr[corr['corr'] >= CORR_SIGNIFICANT][
        ['feature_1', 'feature_2']
    ].to_numpy()
    similar_pairs = set([(a, b) for a, b in correlations])
    similar_pairs.update([(b, a) for a, b in correlations])

    columns = []
    for feature in best_features:
        # Make pairs with existing columns
        candidates = [
            col for col in columns 
            if (feature, col) in similar_pairs
        ]        
        # Append only if not correlation detected
        if len(candidates) == 0:
            columns.append(feature)

    # Limit to 3 features
    columns = columns[:n]
    print('Best features:', columns)
    return columns


def plot_scores(metric_ranks, n=None):
    fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    for i, col in enumerate(('corr', 'f_stat', 'mi')):
        scores = metric_ranks[col].sort_values(ascending=False)
        if n is not None:
            scores = scores.iloc[:n]
        ax[i].bar(scores.index, scores)

    for i, col_name in enumerate(('Correlation', 'F statistic', 'Mutual information')):
        ax[i].set_xticks(ax[i].get_xticks())
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=45, ha='right')
        ax[i].grid()
        ax[i].set_xlabel('Feature')
        ax[i].set_ylabel(col_name)


def ensemble_feature_ranking(scores: pd.DataFrame):
    ranks = scores.rank(axis='rows', method='first', ascending=False)
    return ranks.apply(gmean, axis=1).sort_values().to_frame(name='rank') # Rank product


def rank_correlation(scores: pd.DataFrame):
    # spearman's rho vs kendall's tau
    ranks = scores.rank(axis='rows', method='first', ascending=False)
    correlations = []
    for a, b in itertools.combinations(ranks.columns, r=2):
        coef, pval = spearmanr(ranks[a], ranks[b])
        # coef, pval = kendalltau(ranks[a], ranks[b])
        correlations.append({'A': a, 'B': b, 'Spearman': coef, 'P-value': pval})

    return pd.DataFrame.from_records(correlations)

Online feature selection

In [ ]:
def calculate_scores_online(x: pd.DataFrame, y: pd.DataFrame, metric) -> pd.DataFrame:
    selector = feature_selection.SelectKBest(similarity=metric(), k=2)
    for xs, ys in stream.iter_pandas(x, y):
        selector.learn_one(xs, ys)
        
    leaderboard = (
        pd.DataFrame(selector.leaderboard.items(), columns=['feature', 'score'])
          .set_index('feature')
          .sort_values(by='score', ascending=False)
    )
    return leaderboard


def features_subset_online(
        filename: str, 
        classes: Dict[str, str],
        axis: Tuple[str], 
        label: str,
        train_size: float, 
        anomaly_severity: float,
        balance: bool,
        rpm_limit: bool
    ):
    features = pd.read_csv(filename)
    features = features[features['fault'].isin(classes)]
    if rpm_limit:
        RPM = 2900
        RPM_RANGE = 500
        features = features[features['rpm'].between(RPM - RPM_RANGE, RPM + RPM_RANGE, inclusive='both')]
    
    features = models.fault_labeling(features, classes, anomaly_severity)

    groups = [
        df.sample(frac=1, random_state=10)
        for i, df in (
            features.sort_values(by='severity_level').groupby('severity_level')
        )
    ]
    features = pd.concat(groups).reset_index(drop=True)

    columns = features.columns.str.startswith(tuple(axis))
    X = features[features.columns[columns]].fillna(0)
    y = features[label].astype('category')
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_size, random_state=10
    )   
    return (
        X_train.sort_index(), y_train.sort_index(),
        X_test.sort_index(), y_test.sort_index()
    )

def plot_online_best_features(x: pd.DataFrame, y: pd.Series, title: str, metric: Callable) -> pd.DataFrame:
    selector = feature_selection.SelectKBest(similarity=metric(), k=2)

    best = []
    for xs, ys in stream.iter_pandas(x, y):
        selector.learn_one(xs, ys)
        best.append({k: abs(v) for k, v in selector.leaderboard.items()})

    # Get only n best featues to plot
    n = 15
    n_top_names = [(k, abs(v)) for k, v in selector.leaderboard.items()]
    n_top_names = sorted(n_top_names, key=lambda x: x[1], reverse=True)[:n]
    n_top_names = set(map(lambda x: x[0], n_top_names))
    best = [
        {k: v for k, v in step.items() if k in n_top_names}
        for step in best
    ]

    feature_set = pd.DataFrame.from_records(best)
    kwargs = dict(figsize=(15, 6), grid=True, xlabel='Observation', ylabel=title)
    if metric == selection.FisherScore:
        kwargs['ylim'] = (0, 2 * feature_set.describe().max(axis=1)['75%']) 
    feature_set.plot(**kwargs)

    return feature_set


def plot_online_best_ensemble_features(x: pd.DataFrame, y: pd.Series, title: str) -> pd.DataFrame:
    estimators = [
        feature_selection.SelectKBest(similarity=metric(), k=2)
        for metric in METRICS_ONLINE
    ]
    rating = preprocessing.MinMaxScaler()

    best = []
    for xs, ys in stream.iter_pandas(x, y):
        for method in estimators:
            method.learn_one(xs, ys)

        # TODO: Remove correlated features

        scores = [method.leaderboard.copy() for method in estimators]
        scores = pd.DataFrame.from_records(scores).T
        ranks = scores.rank(axis='rows', method='first', ascending=False)
        ranks = ranks.apply(gmean, axis=1).to_dict()   # Smallest rank is the best

        # Scale ranks to interval (0, 1) - worst to best
        values = [{'x': x} for x in ranks.values()]
        for value in values:
            rating.learn_one(value)

        ranks = {k: 1 - rating.transform_one({'x': v})['x'] for k, v in ranks.items()}
        best.append(ranks)   

    # Get only n best featues to plot
    n = 15
    n_top_names = [(k, abs(v)) for k, v in best[-1].items()]
    n_top_names = sorted(n_top_names, key=lambda x: x[1], reverse=True)[:n]
    n_top_names = set(map(lambda x: x[0], n_top_names))
    best = [
        {k: v for k, v in step.items() if k in n_top_names}
        for step in best
    ]                       

    feature_set = pd.DataFrame.from_records(best)
    feature_set.plot(figsize=(15, 6), grid=True, xlabel='Observation', ylabel=title)

    return feature_set

Feature EDA

In [ ]:
def show_effect_of_normalization(feature_set: str, axis: list, metric: Callable):
    x, y, _, _ = load_feature_set(feature_set, axis, None)
    scores = calculate_scores(x, y, metric)
    
    features_normalized = selection.normalize_features(x, x.columns)
    scores_norm = calculate_scores(features_normalized, y, metric)
    
    fig, ax = plt.subplots(1, 2, figsize=(20, 5))
    scores.head(15).plot.bar(figsize=(10, 4), grid=True, xlabel='Feature', ylabel='Metric', legend=False, title='Unnormalized', ax=ax[0])
    scores_norm.head(15).plot.bar(figsize=(10, 4), grid=True, xlabel='Feature', ylabel='Metric', legend=False, title='Normalized', ax=ax[1])
    plt.show()


def scatter_features(x_train, y_train, x_test, y_test, columns, target):
    train_data = x_train[columns].copy()
    test_data = x_test[columns].copy()

    scaler = MinMaxScaler()
    train_data[columns] = scaler.fit_transform(train_data)
    test_data[columns] = scaler.transform(test_data)

    print('[Train] Silhouette coefficient: ', silhouette_score(train_data, y_train))
    print('[Test] Silhouette coefficient: ', silhouette_score(test_data, y_test))

    if target == 'fault':
        models.cross_cuts_3d(train_data, y_train)
        plt.show()
        models.cross_cuts_3d(test_data, y_test)
        plt.show()

    elif target == 'anomaly':
        models.cross_cuts_3d_anomalies(train_data, y_train)
        plt.show()
        models.cross_cuts_3d_anomalies(test_data, y_test)
        plt.show()


def pca_scatter(x_train, y_train, x_test, y_test, target):
    y_train = y_train.reset_index(drop=True).astype('category')
    y_test = y_test.reset_index(drop=True).astype('category')

    scaler = MinMaxScaler()
    x_train[x_train.columns] = scaler.fit_transform(x_train)
    x_test[x_test.columns] = scaler.transform(x_test)

    model = PCA(n_components=3).fit(x_train)
    x_train = pd.DataFrame(model.transform(x_train))
    x_test = pd.DataFrame(model.transform(x_test))

    print('[PCA Train] Silhouette coefficient: ', silhouette_score(x_train, y_train))
    print('[PCA Test] Silhouette coefficient: ', silhouette_score(x_test, y_test))
    
    print('PC explained variance:', model.explained_variance_ratio_)
    if target == 'fault':
        models.cross_cuts_3d(x_train, y_train)
        plt.show()
        models.cross_cuts_3d(x_test, y_test)
        plt.show()

    elif target == 'anomaly':
        models.cross_cuts_3d_anomalies(x_train, y_train)
        plt.show()
        models.cross_cuts_3d_anomalies(x_test, y_test)
        plt.show()


def pca_feature_importance(X, n=10):
    # Absolute values of the Eigenvectors' components corresponding to the k largest Eigenvalues.
    model = PCA(n_components=3).fit(X)
    X_pc = model.transform(X)

    columns = list(X.columns)
    percentages = [(100 * (np.flip(np.sort(np.abs(pc))) / np.sum(np.abs(pc))))[:n] for pc in model.components_]
    most_important = [np.flip(np.argsort(np.abs(pc)))[:n] for pc in model.components_]

    for i, pc in enumerate(most_important):
        print(f'PC{i+1} ({model.explained_variance_ratio_[i] * 100:.4f} %)')
        print([columns[x] for x in pc])
        print(percentages[i])


def plot_label_occurences(y):
    observations = []
    y = y.astype('category')
    columns = list(y.cat.categories)
    empty = dict(zip(columns, len(columns) * [0]))

    for row in y:
        sample = empty.copy()
        sample[row] = 1
        observations.append(sample)

    class_occurences = pd.DataFrame.from_records(observations).cumsum()
    class_occurences.plot(grid=True, figsize=(10, 3), xlabel='Observations', ylabel='Label occurences')

Feature distribution

In [ ]:
x, y, _, _ = load_feature_set(TD_FEATURES, PLACE, None)
ax = x.boxplot(figsize=(20, 5))
plt.show()

In [ ]:
scaler = MinMaxScaler()
x[x.columns] = scaler.fit_transform(x)
ax = x.boxplot(figsize=(20, 5))
plt.show()

In [ ]:
# Normalize
scaler = StandardScaler()
x[x.columns] = scaler.fit_transform(x)
ax = x.boxplot(figsize=(20, 5))
plt.show()

In [ ]:
x, y, _, _ = load_feature_set(FD_FEATURES, PLACE, 2**14)
ax = x.boxplot(figsize=(20, 5))
plt.show()

In [ ]:
scaler = MinMaxScaler()
x[x.columns] = scaler.fit_transform(x)
ax = x.boxplot(figsize=(20, 5))
plt.show()

In [ ]:
ax = x.plot.hist(figsize=(10, 20), subplots=True, bins=100, grid=True)
plt.show()

### Show effect of normalization on metrics
Result: normalization has no effect

In [ ]:
show_effect_of_normalization(TD_FEATURES, PLACE, METRICS_OFFLINE[0])

In [ ]:
show_effect_of_normalization(TD_FEATURES, PLACE, METRICS_OFFLINE[1])

In [ ]:
show_effect_of_normalization(TD_FEATURES, PLACE, METRICS_OFFLINE[2])

### Show class imbalance

In [ ]:
def plot_class_imbalance(feature_set, axis):
    _, y, _, _ = load_feature_set(feature_set, axis, None)
    counter = Counter(y)
    for k, v in counter.items():
        per = v / len(y) * 100
        print(f'Class={k}, n={v} ({per:.3f}%%)')
    
    plt.bar(counter.keys(), counter.values())
    plt.show()

plot_class_imbalance(TD_FEATURES, PLACE)

In [ ]:
plot_class_imbalance(FD_FEATURES, PLACE)

---
## Feature set #1: Temporal domain features

In [ ]:
corr = corr_among_features(TD_FEATURES, PLACE)
corr.head(10)

In [ ]:
metric_scores = feature_ranking(TD_FEATURES)
metric_scores

In [ ]:
plot_scores(metric_scores)

In [ ]:
rank_correlation(metric_scores)

In [ ]:
ranks = ensemble_feature_ranking(metric_scores)
ranks

In [ ]:
x_train, y_train, x_test, y_test = load_feature_set(TD_FEATURES, PLACE, None)
columns = extract_best_features(x_train, y_train, list(ranks.index), corr)
scatter_features(x_train, y_train, x_test, y_test, columns, VAR_TARGET)
plt.show()

In [ ]:
# Compare feature selection to PCA
pca_scatter(x_train, y_train, x_test, y_test, VAR_TARGET)

In [ ]:
pca_feature_importance(x_train)

In [ ]:
# Online learning
x, y, _, _ = load_feature_set(TD_FEATURES, PLACE, None, online=True)
plot_label_occurences(y)

In [ ]:
# Online learning (Correlation)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[0], METRICS_ONLINE[0])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[0]})
    .sort_values(by=METRICS_TITLES[0], ascending=False))

In [ ]:
# Online learning (F statistic)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[1], METRICS_ONLINE[1])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[1]})
    .sort_values(by=METRICS_TITLES[1], ascending=False))

In [ ]:
# 3) Online learning (Mutual information)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[2], METRICS_ONLINE[2])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[2]})
    .sort_values(by=METRICS_TITLES[2], ascending=False))

In [ ]:
title = 'Rank product ensemble'
online_feature_set = plot_online_best_ensemble_features(x, y, title)
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: title})
    .sort_values(by=title, ascending=False))

---
## Feature set #2: Spectral features

In [ ]:
c = corr_among_features(FD_FEATURES, PLACE)
c.head(20)

In [ ]:
metric_scores = feature_ranking(FD_FEATURES)
metric_scores

In [ ]:
plot_scores(metric_scores, n=15)

In [ ]:
rank_correlation(metric_scores)

In [ ]:
ranks = ensemble_feature_ranking(metric_scores)
ranks.head(10)

In [ ]:
x_train, y_train, x_test, y_test = load_feature_set(FD_FEATURES, PLACE, None)
columns = extract_best_features(x_train, y_train, list(ranks.index), corr)
scatter_features(x_train, y_train, x_test, y_test, columns, VAR_TARGET)
plt.show()

In [ ]:
pca_scatter(x_train, y_train, x_test, y_test, VAR_TARGET)

In [ ]:
# Online learning
x, y, _, _ = load_feature_set(FD_FEATURES, PLACE, None, online=True)
plot_label_occurences(y)

In [ ]:
# Online learning (Correlation)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[0], METRICS_ONLINE[0])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[0]})
    .sort_values(by=METRICS_TITLES[0], ascending=False))

In [ ]:
# Online learning (F statistic)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[1], METRICS_ONLINE[1])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[1]})
    .sort_values(by=METRICS_TITLES[1], ascending=False))

In [ ]:
# 3) Online learning (Mutual information)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[2], METRICS_ONLINE[2])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[2]})
    .sort_values(by=METRICS_TITLES[2], ascending=False))

In [ ]:
title = 'Rank product ensemble'
online_feature_set = plot_online_best_ensemble_features(x, y, title)
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: title})
    .sort_values(by=title, ascending=False))

## Frequency domain: window size = 64

In [ ]:
window_sizes = (2**6, 2**8, 2**10, 2**12, 2**14)

win_len = window_sizes[0]
print('Window size:', win_len)
metric_scores = feature_ranking(FD_FEATURES, win_len)
metric_scores

In [ ]:
plot_scores(metric_scores)

In [ ]:
print('Window size:', win_len)
ranks = ensemble_feature_ranking(metric_scores)
ranks

In [ ]:
corr = corr_among_features(FD_FEATURES, PLACE, win_len)
corr

In [ ]:
x_train, y_train, x_test, y_test = load_feature_set(FD_FEATURES, PLACE, win_len)
columns = extract_best_features(x_train, y_train, list(ranks.index), corr)
scatter_features(x_train, y_train, x_test, y_test, columns, VAR_TARGET)
plt.show()

In [ ]:
pca_scatter(x_train, y_train, x_test, y_test, VAR_TARGET)

In [ ]:
# Online learning
x, y, _, _ = load_feature_set(FD_FEATURES, PLACE, None, online=True)
plot_label_occurences(y)

In [ ]:
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[0], METRICS_ONLINE[0])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[0]})
    .sort_values(by=METRICS_TITLES[0], ascending=False))

In [ ]:
# Online learning (F statistic)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[1], METRICS_ONLINE[1])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[1]})
    .sort_values(by=METRICS_TITLES[1], ascending=False))

In [ ]:
# 3) Online learning (Mutual information)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[2], METRICS_ONLINE[2])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[2]})
    .sort_values(by=METRICS_TITLES[2], ascending=False))

In [ ]:
title = 'Rank product ensemble'
online_feature_set = plot_online_best_ensemble_features(x, y, title)
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: title})
    .sort_values(by=title, ascending=False))

---
## Frequency domain: window size = 256

In [ ]:
win_len = window_sizes[1]
print('Window size:', win_len)
metric_ranks = feature_ranking(FD_FEATURES, win_len)
metric_ranks

In [ ]:
plot_scores(metric_ranks)

In [ ]:
rank_correlation(metric_scores)

In [ ]:
ranks = ensemble_feature_ranking(metric_scores)
ranks

In [ ]:
corr = corr_among_features(FD_FEATURES, PLACE, win_len)
corr

In [ ]:
x_train, y_train, x_test, y_test = load_feature_set(FD_FEATURES, PLACE, win_len)
columns = extract_best_features(x_train, y_train, list(ranks.index), corr)
scatter_features(x_train, y_train, x_test, y_test, columns, VAR_TARGET)
plt.show()

In [ ]:
pca_scatter(x_train, y_train, x_test, y_test, VAR_TARGET)

In [ ]:
# Online learning
x, y, _, _ = load_feature_set(FD_FEATURES, PLACE, win_len, online=True)
plot_label_occurences(y)

In [ ]:
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[0], METRICS_ONLINE[0])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[0]})
    .sort_values(by=METRICS_TITLES[0], ascending=False))

In [ ]:
# Online learning (F statistic)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[1], METRICS_ONLINE[1])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[1]})
    .sort_values(by=METRICS_TITLES[1], ascending=False))

In [ ]:
# 3) Online learning (Mutual information)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[2], METRICS_ONLINE[2])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[2]})
    .sort_values(by=METRICS_TITLES[2], ascending=False))

In [ ]:
title = 'Rank product ensemble'
online_feature_set = plot_online_best_ensemble_features(x, y, title)
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: title})
    .sort_values(by=title, ascending=False))

---
## Frequency domain: window size = 1024

In [ ]:
win_len = window_sizes[2]
print('Window size:', win_len)
metric_scores = feature_ranking(FD_FEATURES, win_len)
metric_scores

In [ ]:
plot_scores(metric_scores)

In [ ]:
rank_correlation(metric_scores)

In [ ]:
ranks = ensemble_feature_ranking(metric_scores)
ranks

In [ ]:
corr = corr_among_features(FD_FEATURES, PLACE, win_len)
corr

In [ ]:
x_train, y_train, x_test, y_test = load_feature_set(FD_FEATURES, PLACE, win_len)
columns = extract_best_features(x_train, y_train, list(ranks.index), corr)
scatter_features(x_train, y_train, x_test, y_test, columns, VAR_TARGET)
plt.show()

In [ ]:
pca_scatter(x_train, y_train, x_test, y_test, VAR_TARGET)

In [ ]:
# Online learning
x, y, _, _ = load_feature_set(FD_FEATURES, PLACE, win_len, online=True)
plot_label_occurences(y)

In [ ]:
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[0], METRICS_ONLINE[0])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[0]})
    .sort_values(by=METRICS_TITLES[0], ascending=False))

In [ ]:
# Online learning (F statistic)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[1], METRICS_ONLINE[1])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[1]})
    .sort_values(by=METRICS_TITLES[1], ascending=False))

In [ ]:
# 3) Online learning (Mutual information)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[2], METRICS_ONLINE[2])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[2]})
    .sort_values(by=METRICS_TITLES[2], ascending=False))

In [ ]:
title = 'Rank product ensemble'
online_feature_set = plot_online_best_ensemble_features(x, y, title)
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: title})
    .sort_values(by=title, ascending=False))

---
## Frequency domain: window size = 4096

In [ ]:
win_len = window_sizes[3]
print('Window size:', win_len)
metric_scores = feature_ranking(FD_FEATURES, win_len)
metric_scores

In [ ]:
plot_scores(metric_scores)

In [ ]:
rank_correlation(metric_scores)

In [ ]:
ranks = ensemble_feature_ranking(metric_scores)
ranks

In [ ]:
corr = corr_among_features(FD_FEATURES, PLACE, win_len)
corr

In [ ]:
x_train, y_train, x_test, y_test = load_feature_set(FD_FEATURES, PLACE, win_len)
columns = extract_best_features(x_train, y_train, list(ranks.index), corr)
scatter_features(x_train, y_train, x_test, y_test, columns, VAR_TARGET)
plt.show()

In [ ]:
pca_scatter(x_train, y_train, x_test, y_test, VAR_TARGET)

In [ ]:
# Online learning
x, y, _, _ = load_feature_set(FD_FEATURES, PLACE, win_len, online=True)
plot_label_occurences(y)

In [ ]:
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[0], METRICS_ONLINE[0])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[0]})
    .sort_values(by=METRICS_TITLES[0], ascending=False))

In [ ]:
# Online learning (F statistic)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[1], METRICS_ONLINE[1])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[1]})
    .sort_values(by=METRICS_TITLES[1], ascending=False))

In [ ]:
# 3) Online learning (Mutual information)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[2], METRICS_ONLINE[2])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[2]})
    .sort_values(by=METRICS_TITLES[2], ascending=False))

In [ ]:
title = 'Rank product ensemble'
online_feature_set = plot_online_best_ensemble_features(x, y, title)
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: title})
    .sort_values(by=title, ascending=False))

---

## Frequency domain: window size = 16384

In [ ]:
win_len = window_sizes[4]
print('Window size:', win_len)
metric_scores = feature_ranking(FD_FEATURES, win_len)
metric_scores

In [ ]:
plot_scores(metric_scores)

In [ ]:
rank_correlation(metric_scores)

In [ ]:
ranks = ensemble_feature_ranking(metric_scores)
ranks

In [ ]:
corr = corr_among_features(FD_FEATURES, PLACE, win_len)
corr.head(10)

In [ ]:
x_train, y_train, x_test, y_test = load_feature_set(FD_FEATURES, PLACE, win_len)
columns = extract_best_features(x_train, y_train, list(ranks.index), corr)
scatter_features(x_train, y_train, x_test, y_test, columns, VAR_TARGET)
plt.show()

In [ ]:
pca_scatter(x_train, y_train, x_test, y_test, VAR_TARGET)

In [ ]:
# Online learning
x, y, _, _ = load_feature_set(FD_FEATURES, PLACE, win_len, online=True)
plot_label_occurences(y)

In [ ]:
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[0], METRICS_ONLINE[0])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[0]})
    .sort_values(by=METRICS_TITLES[0], ascending=False))

In [ ]:
# Online learning (F statistic)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[1], METRICS_ONLINE[1])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[1]})
    .sort_values(by=METRICS_TITLES[1], ascending=False))

In [ ]:
# 3) Online learning (Mutual information)
online_feature_set = plot_online_best_features(x, y, METRICS_TITLES[2], METRICS_ONLINE[2])
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: METRICS_TITLES[2]})
    .sort_values(by=METRICS_TITLES[2], ascending=False))

In [ ]:
title = 'Rank product ensemble'
online_feature_set = plot_online_best_ensemble_features(x, y, title)
plt.show()

(online_feature_set.tail(1)
    .reset_index(drop=True)
    .T.rename(columns={0: title})
    .sort_values(by=title, ascending=False))

#### Evolution of ranks depending in window size (scatter)

In [ ]:
feature_window_ranks = pd.DataFrame()
for win in window_sizes:
    scores = feature_ranking(FD_FEATURES, win)
    ranks = ensemble_feature_ranking(scores)
    feature_window_ranks[win] = ranks

feature_window_ranks.sort_values(by=1024)